# Visualize a filter

In [2]:
from scipy import ndimage, misc
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet, imagenet_utils
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
#import tqdm

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\casti\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\casti\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\casti\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\casti\AppData\Local\Continuum\anaconda3\envs\tf_test\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\casti\AppData\Local\Continuum\anaconda3\envs\tf_test\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Das angege

TypeError: can only concatenate str (not "list") to str

In [ ]:
def process_for_model(file):
    image = cv2.imread(file)
    image = cv2.resize(image, dsize=(1000,1000),interpolation=cv2.INTER_CUBIC)
#    image = preprocess_input(image)
    return image

In [ ]:
corona = process_for_model('C:/Users/casti/Documents/COVID-19-imaging_analysis_Juan/keras-covid-19/keras-covid-19/dataset/covid/1-s2.0-S0929664620300449-gr2_lrg-a.jpg')

In [ ]:
fig = plt.figure(figsize=(15, 15))
plt.gray()
plt.imshow(corona)

In [ ]:
fig = plt.figure(figsize=(15, 15))
plt.gray()  # show the filtered result in grayscale
ax1 = fig.add_subplot(121)  # left side
ax2 = fig.add_subplot(122)  # right side
ascent = corona
result = ndimage.sobel(corona, mode='constant', cval=0.0)
ax1.imshow(corona)
ax2.imshow(result)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 15))
plt.gray()  # show the filtered result in grayscale
ax1 = fig.add_subplot(121)  # left side
ax2 = fig.add_subplot(122)  # right side
ascent = corona
result = ndimage.sobel(corona, mode='constant', cval=0.0, axis=0)
ax1.imshow(corona)
ax2.imshow(result)
plt.show()

# Build a Model

In [ ]:
# load the necessary modules
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

In [ ]:
X = []
y = []
path = 'C:/Users/casti/Documents/COVID-19-imaging_analysis_Juan/keras-covid-19/both/'

train_data = os.listdir(path)

for picture in train_data:
    image = process_for_model(path+picture)
    X.append(image)
    if 'corona' in picture:
        y.append(0)
    else:
        y.append(1) 
X = np.array(X)
y = np.array(y)

In [ ]:
y

In [ ]:
# load the data
xtrain, xtest, ytrain, ytest = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
# inspect the data
plt.figure(figsize=(12,20))
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(xtrain[i], cmap=plt.cm.Greys)
    plt.axis('off')

In [ ]:
xtrain.shape

In [ ]:
xtest.shape

In [ ]:
# reshape and normalize the data
xtrain = xtrain.reshape(40, 254, 254, 3)
xtest = xtest.reshape(10, 254, 254, 3)

In [ ]:
# convert output to categorical
ytrain = to_categorical(ytrain)
ytest = to_categorical(ytest)

In [ ]:
xtest

In [ ]:
ytest

In [ ]:
# define the model
baseModel = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(254, 254, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
    layer.trainable = True

In [ ]:
model.summary()

In [ ]:
# compile the model


In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
results = model.fit(xtrain, ytrain, epochs=5, batch_size=4, validation_split=0.2)

In [ ]:
# plot the accuracy
plt.figure()
plt.plot(np.arange(0, 5), results.history["loss"], label="train_loss")
plt.plot(np.arange(0, 5), results.history["accuracy"], label="train_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
#plt.savefig(args["plot"])

### Let's have a look at the feature maps

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import expand_dims

# redefine model to output right after the first hidden layer
model_small = Model(inputs=model.inputs, outputs=model.layers[1].output)
model_small.summary()

In [ ]:
# load the image with the required shape
img = xtrain[0]
plt.imshow(img.reshape(28,28), cmap=plt.cm.Greys)

In [ ]:
# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
img.shape

In [ ]:
# get feature map for first hidden layer
feature_maps = model_small.predict(img)
feature_maps[0]

In [ ]:
# plot all 32 maps in an 4*8 squares
height = 4
width = 8
ix = 1
plt.figure(figsize=(12,4))
for _ in range(height):
    for _ in range(width):
        # specify subplot and turn of axis
        ax = plt.subplot(height, width, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
        ix += 1
# show the figure
plt.show()

## Further readings

#### Stanford Ressource

https://cs231n.github.io/convolutional-networks/

#### Dive into the math behind convolutional neural networks

https://towardsdatascience.com/gentle-dive-into-math-behind-convolutional-neural-networks-79a07dd44cf9

## Visualization

#### Visualization toolbox

https://www.youtube.com/watch?v=AgkfIQ4IGaM

#### How to visualize filters in cnns

https://machinelearningmastery.com/how-to-visualize-filters-and-feature-maps-in-convolutional-neural-networks/

#### How to visualize filters in deep layers

https://www.researchgate.net/profile/Aaron_Courville/publication/265022827_Visualizing_Higher-Layer_Features_of_a_Deep_Network/links/53ff82b00cf24c81027da530.pdf